Wstępny kod z funkcjami, ktorych nie mozemy uzywac, zeby zobaczyc czy wyjdzie

In [36]:
import cv2
import numpy as np

# Ładowanie wszystkich 5 obrazów i usuwanie ludzi
images = []
for i in range(1, 6):
    image_path = f"cathedral/d00{i}.jpg"
    img = cv2.imread(image_path)
    images.append(img)

# Obliczanie współrzędnych punktów, w których obrazy zostaną nakładane na siebie
height, width, _ = images[0].shape
combined_image = np.zeros((height, width, 3), dtype=np.uint8)
kernel = np.ones((4, 4), np.uint8) 


# Znajdowanie najczęstszego piksela dla każdego piksela pomiędzy obrazami
for y in range(height):
    for x in range(width):
        pixel_values = [tuple(img[y, x]) for img in images]
        most_frequent_value = max(set(pixel_values), key=pixel_values.count)
        combined_image[y, x] = most_frequent_value
print('good')
# Calculate histogram of the combined image
histogram = cv2.calcHist([combined_image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# Find the peak(s) in the histogram
peaks = np.where(histogram > np.max(histogram) * 0.007) #Mozna pobawic sie jak duze peaki ma wylapywac zmieniajac liczbe ktora mnozymy, dobre sa z zakresu 0.01 - 0.004

# Customized mask based on color similarity 
# testowa maska ktora probuje brac pod uwage podobienstwo kolorow, bardzo powolna i nawet nie zdazylem jej odpalic

mask = np.zeros((height, width), dtype=np.uint8)

# Threshold the image based on the peak values 
#prosta maska ktora tylko usuwa peaki, dziala okej
mask = np.zeros((height, width), dtype=np.uint8)
for i in range(len(peaks[0])):
    mask[np.where((combined_image[:, :, 0] == peaks[0][i]) & (combined_image[:, :, 1] == peaks[1][i]) & (combined_image[:, :, 2] == peaks[2][i]))] = 255

# Apply the mask to remove unwanted pixels
result_image = cv2.bitwise_and(combined_image, combined_image, mask=mask)
#result_image_dil_mean = cv2.medianBlur(cv2.dilate(result_image,kernel),3) #replace this function with mine
custom_kernel_size = 4
tolerance = 35
#najlepiej dla tolerance 35, kernel 4
result_image_dil_mean = custom_median_dilate(result_image, custom_kernel_size, tolerance= tolerance)
#for i in range(8):
#    result_image_dil_mean = custom_median_dilate(result_image_dil_mean, custom_kernel_size)
split=350
vis = np.concatenate((images[0][:split,:,:], result_image_dil_mean[split:,:,:]), axis=0)

# Inpainting to replace removed pixels  #ten inpainting slabo mi dzialal takze trzeba zrobic funkcje do wypelniania
#result_image = cv2.inpaint(result_image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Zapisanie obrazu bez ludzi
# cv2.imwrite("combined_no_people_overlaid.jpg", cv2.filter2D(cv2.dilate(result_image,kernel), -1,kernel/np.sum(kernel)))

cv2.imwrite("combined_no_people_overlaid.jpg", vis)

good


True

In [31]:
def custom_median_dilate(image, kernel_size, tolerance=30):
    height, width, _ = image.shape
    result = np.copy(image)  # Create a copy of the original image

    for _ in range(5):  # Repeat the process multiple times to ensure all black pixels are replaced
        for y in range(height):
            for x in range(width):
                if np.all(np.abs(image[y, x] - [0, 0, 0]) <= tolerance):  # If black pixel within tolerance range
                    non_black_pixels = []
                    for i in range(-kernel_size, kernel_size + 1):
                        for j in range(-kernel_size, kernel_size + 1):
                            if 0 <= y + i < height and 0 <= x + j < width and not np.all(np.abs(image[y + i, x + j] - [0, 0, 0]) <= tolerance):
                                non_black_pixels.append(image[y + i, x + j])
                    if non_black_pixels:  # If there are non-black pixels around
                        result[y, x] = np.mean(non_black_pixels, axis=0).astype(np.uint8)
                    #else:  # If no non-black pixels around, retain black pixel
                        #result[y, x] = [0, 0, 0]
                # else:  # If pixel is not black within tolerance range, retain its original value
                    # result[y, x] = image[y, x]  # This line is commented out as it's unnecessary
        image = np.copy(result)  # Update the image for the next iteration

    return result

###Od tego momentu eksperymenty Przemka

In [37]:
def custom_dilate_with_blur(image, mask, kernel_size, tolerance):
    height, width, _ = image.shape
    result = np.copy(image)  # Create a copy of the original image
    
    # Create a kernel for blurring
    blur_kernel = np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size ** 2)

    for y in range(height):
        for x in range(width):
            if mask[y, x] == 255:  # If pixel is in the mask (unwanted)
                neighborhood = image[max(0, y - kernel_size):min(height, y + kernel_size + 1),
                                     max(0, x - kernel_size):min(width, x + kernel_size + 1)]
                if np.all(np.abs(neighborhood - [0, 0, 0]) > tolerance):  # If pixel is not black within tolerance range
                    blurred_pixel = cv2.filter2D(neighborhood, -1, blur_kernel)[kernel_size // 2, kernel_size // 2]
                    result[y, x] = blurred_pixel

    return result

In [40]:
def custom_median_dilate_with_kernel_and_mask(image, mask, kernel_size, tolerance):
    height, width, _ = image.shape
    result = np.copy(image)  # Create a copy of the original image
    
    # Create a histogram-like kernel to filter out peaks
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32)
    kernel[kernel_size // 2, kernel_size // 2] = 0  # Center of the kernel is set to 0
    
    # Apply the kernel to the image
    filtered_image = cv2.filter2D(image, -1, kernel)
    
    for y in range(height):
        for x in range(width):
            if mask[y, x] == 255:  # If pixel is in the mask (unwanted)
                neighborhood = filtered_image[max(0, y - kernel_size // 2):min(height, y + kernel_size // 2 + 1),
                                              max(0, x - kernel_size // 2):min(width, x + kernel_size // 2 + 1)]
                if np.all(np.abs(neighborhood - [0, 0, 0]) > tolerance):  # If pixel is not black within tolerance range
                    non_black_pixels = neighborhood[np.any(neighborhood != [0, 0, 0], axis=-1)]
                    if non_black_pixels.any():  # If there are non-black pixels around
                        result[y, x] = np.mean(non_black_pixels, axis=0).astype(np.uint8)
                    else:  # If no non-black pixels around, retain black pixel
                        result[y, x] = [0, 0, 0]

    return result

# Usage:
custom_kernel_size = 5
tolerance = 5
result_image_with_kernel_and_mask = custom_median_dilate_with_kernel_and_mask(combined_image, mask, custom_kernel_size, tolerance)

# Split and concatenate the images as before
split = 350
vis = np.concatenate((images[0][:split,:,:], result_image_with_kernel_and_mask[split:,:,:]), axis=0)

# Save the result
cv2.imwrite("combined_no_people_overlaid_with_kernel_and_mask.jpg", vis)

True